In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [61]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import E8257D

In [48]:
exa = Agilent_EXA_N9010A("SA", "EXA")
awg = KeysightAWG("AWG2")
mwsrc = E8257D("MXG")

In [55]:
reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

In [63]:
cal = IQCalibrator(awg, exa, mwsrc, "TEST", -20)
res = cal.calibrate(lo_frequency=5e9, if_frequency=100e6, lo_power=10,
                    ssb_power=-20, iterations=5, minimize_iterlimit=20,sa_res_bandwidth=1000,
                    initial_guess=res.get_optimization_results()[0])

Phase:  [-1.96471537] [-15.593219757080078, -81.66436767578125, -104.99742126464844], 

In [44]:
print(res)

Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -20}
Radiation parameters: {'if_frequency': 100000000.0, 'lo_power': 10, 'ssb_power': -20, 'lo_frequency': 5000000000.0}
Optimization results: {'dc': -103.53374481201172, 'if': [-19.828022003173828, -84.92497253417969, -104.45783996582031]}
Optimization time: 0 h 4 m 10.82 s


In [11]:
res.get_optimization_results()

({'dc_offsets': array([-0.08081103, -0.03753255]),
  'if_amplitudes': array([ 1.34394893,  1.47900162]),
  'if_offsets': array([-0.09390972, -0.04513448]),
  'if_phase': array([-3.90144169])},
 {'dc': -96.877677917480469,
  'if': [-20.11989402770996, -91.63507080078125, -112.01741790771484]})

In [71]:
exa.setup_swept_sa(4.9e9, 500e6, nop=1001, rbw=1e5)
exa.set_continuous()

In [39]:
reload(dm)

<module 'lib.data_management' from 'C:\\Users\\labiks\\Documents\\Gleb scripts\\lib\\data_management.py'>

In [69]:
dm.save_IQMX_calibration(res)

In [16]:
res.get_radiation_parameters()

{'if_frequency': 100000000.0,
 'lo_frequency': 5000000000.0,
 'lo_power': 10,
 'ssb_power': -20}

In [28]:
list(dm.load_IQMX_calibration_database("TEST", -20).values())[0].get_optimization_results()

({'dc_offsets': array([-0.08081103, -0.03753255]),
  'if_amplitudes': array([ 1.34394893,  1.47900162]),
  'if_offsets': array([-0.09390972, -0.04513448]),
  'if_phase': array([-3.90144169])},
 {'dc': -96.877677917480469,
  'if': [-20.11989402770996, -91.63507080078125, -112.01741790771484]})

In [41]:
list(dm.load_IQMX_calibration_database("TEST", -20).keys())

[frozenset({('if_frequency', 100000000.0),
            ('lo_frequency', 5000000000.0),
            ('lo_power', 10),
            ('ssb_power', -20)})]